In [72]:
import pickle
import numpy as np
import itertools as it
import matplotlib.pylab as plt
import seaborn as sns
from pprint import pprint
from myutils import *
import math
from scipy.stats import kendalltau

%config InlineBackend.figure_format = 'retina'  # 提高 jupyter notebook 的圖形顯示解析度

In [73]:
def count_discordant_concordant_pairs(x, y):
    n = len(x)
    discordant_pairs = 0
    concordant_pairs = 0
    equal_pairs = 0
    partial_eq_pairs = 0

    for i in range(n):
        for j in range(i+1, n):
            if (x[i] < x[j] and y[i] > y[j]) or (x[i] > x[j] and y[i] < y[j]):
                discordant_pairs += 1
            elif (x[i] < x[j] and y[i] < y[j]) or (x[i] > x[j] and y[i] > y[j]):
                concordant_pairs += 1
            else:
                # print((x[i], y[i]), (x[j], y[j]))
                if math.isclose(x[i], x[j]) and math.isclose(y[i], y[j]):
                    equal_pairs += 1
                else:
                    partial_eq_pairs += 1
    
    def bucketize(value, bucket_size=1e-9):
        return round(value / bucket_size)
    
    def count_tied_values(arr):
        counts = {}
        for val in arr:
            if bucketize(val) in counts:
                counts[bucketize(val)] += 1
            else:
                counts[bucketize(val)] = 1
        return counts
    
    def calculate_tied_pairs(counts):
        tied_pairs = sum([(count * (count - 1)) // 2 for count in counts.values()])
        return tied_pairs
    
    counts = count_tied_values(x)
    tied_pairs_x = calculate_tied_pairs(counts)
    # print('x_counts:', len(counts), counts)
    # print('tied_pairs_x:', tied_pairs_x)
    
    counts = count_tied_values(y)
    tied_pairs_y = calculate_tied_pairs(counts)
    # print('y_counts:', len(counts), counts)
    # print('y_tied_pairs:', tied_pairs_y)

    return discordant_pairs, concordant_pairs, equal_pairs, partial_eq_pairs, tied_pairs_x, tied_pairs_y, n*(n-1)//2

def calc_tau(discordant_pairs, concordant_pairs, equal_pairs, partial_eq_pairs, T_x, T_y, total_pairs):
    tau = (concordant_pairs - discordant_pairs) / math.sqrt((concordant_pairs + discordant_pairs + T_x) * (concordant_pairs + discordant_pairs + T_y))
    return tau

# BR

In [74]:
dr_res_by_event_filepath = "/home/wmnlab/jackbedford/udp-reliability-research/handover_profiling/20240417_1333007d66_new_data_sync_v2/dr_20240417_1452001e84_anchor_by_event/dl_lost/results/BR_20240417_1636000_b9ecf_iter2.pkl"
dr_res_by_packet_filepath = "/home/wmnlab/jackbedford/udp-reliability-research/handover_profiling/20240417_1333007d66_new_data_sync_v2/dr_20240417_1452001e84_anchor_by_packet/dl_lost/results/BR_20240417_2110000_315b4_iter2.pkl"
sr_res_filepath = "/home/wmnlab/jackbedford/udp-reliability-research/handover_profiling/20240417_1333007d66_new_data_sync_v2/sr/dl_lost/results/BR_20240417_1444000_d7cac_iter5.pkl"
with open(dr_res_by_event_filepath, 'rb') as f:
    dr_res_by_event = pickle.load(f)
with open(dr_res_by_packet_filepath, 'rb') as f:
    dr_res_by_packet = pickle.load(f)
with open(sr_res_filepath, 'rb') as f:
    sr_res = pickle.load(f)

In [75]:
print('======= Single Radio =======')
devices = [item[5] for item in sr_res]
mean_values = [item[1] for item in sr_res]
ground_truths = [item[3] for item in sr_res]

print(devices)
print(ground_truths)
print(mean_values)

tau, p_value = kendalltau(ground_truths, mean_values)
print('tau:', tau)
print('p_value:', p_value)

discordant_pairs, concordant_pairs, equal_pairs, partial_eq_pairs, T_x, T_y, total_pairs = count_discordant_concordant_pairs(ground_truths, mean_values)
tau_ = calc_tau(discordant_pairs, concordant_pairs, equal_pairs, partial_eq_pairs, T_x, T_y, total_pairs)
print('tau self-calc:', tau_)

print("Number of discordant pairs:", discordant_pairs)
print("Number of concordant pairs:", concordant_pairs)
print("Number of equal pairs:", equal_pairs)
print("Number of partial equal pairs:", partial_eq_pairs)
print('Number of tied pairs in X:', T_x)
print('Number of tied pairs in Y:', T_y)
print('Number of total pairs:', total_pairs)
print()

print('======= Dual Radio By Event =======')
devices = [item[5] for item in dr_res_by_event]
mean_values = [item[1] for item in dr_res_by_event]
ground_truths = [item[3] for item in dr_res_by_event]

print(devices)
print(ground_truths)
print(mean_values)

tau, p_value = kendalltau(ground_truths, mean_values)
print('tau:', tau)
print('p_value:', p_value)

discordant_pairs, concordant_pairs, equal_pairs, partial_eq_pairs, T_x, T_y, total_pairs = count_discordant_concordant_pairs(ground_truths, mean_values)
tau_ = calc_tau(discordant_pairs, concordant_pairs, equal_pairs, partial_eq_pairs, T_x, T_y, total_pairs)
print('tau self-calc:', tau_)

print("Number of discordant pairs:", discordant_pairs)
print("Number of concordant pairs:", concordant_pairs)
print("Number of equal pairs:", equal_pairs)
print("Number of partial equal pairs:", partial_eq_pairs)
print('Number of tied pairs in X:', T_x)
print('Number of tied pairs in Y:', T_y)
print('Number of total pairs:', total_pairs)
print()

print('======= Dual Radio By Packet =======')
devices = [item[5] for item in dr_res_by_packet]
mean_values = [item[1] for item in dr_res_by_packet]
ground_truths = [item[3] for item in dr_res_by_packet]

print(devices)
print(ground_truths)
print(mean_values)

tau, p_value = kendalltau(ground_truths, mean_values)
print('tau:', tau)
print('p_value:', p_value)

discordant_pairs, concordant_pairs, equal_pairs, partial_eq_pairs, T_x, T_y, total_pairs = count_discordant_concordant_pairs(ground_truths, mean_values)
tau_ = calc_tau(discordant_pairs, concordant_pairs, equal_pairs, partial_eq_pairs, T_x, T_y, total_pairs)
print('tau self-calc:', tau_)

print("Number of discordant pairs:", discordant_pairs)
print("Number of concordant pairs:", concordant_pairs)
print("Number of equal pairs:", equal_pairs)
print("Number of partial equal pairs:", partial_eq_pairs)
print('Number of tied pairs in X:', T_x)
print('Number of tied pairs in Y:', T_y)
print('Number of total pairs:', total_pairs)
print()

======= Single Radio =======
['sm00', 'sm01', 'sm02', 'sm03', 'sm04', 'sm05', 'sm06', 'sm07', 'sm08', 'sm00', 'sm01', 'sm02', 'sm03', 'sm04', 'sm05', 'sm06', 'sm07', 'sm08']
[1.5687716176356992, 1.6695246499268461, 1.6808411428832972, 1.1174215198048374, 1.2279494745634951, 1.6727438664150913, 1.691899380011171, 1.125760852154413, 0.20866928635533133, 1.431711489139571, 1.4673120318723956, 1.0033978635016543, 0.9990581705502153, 1.3737552993954028, 1.2138422882120559, 1.286694925977477, 1.6771379950695247, 0.18776723763626063]
[1.4205057489679322, 1.2775590391646203, 1.4030729005755609, 1.118541992825635, 1.2471637490053158, 1.4664910024542444, 1.6157327976164508, 0.893834427892573, 0.3089039618321611, 1.3646269763147807, 1.3347260492755935, 0.9863102864338059, 1.2000297167561764, 1.3043192631826097, 1.2799359672551114, 1.370308168359541, 1.6453849108864669, 0.24278428429435847]
tau: 0.7385620915032681
p_value: 2.5215800485929758e-06
tau self-calc: 0.738562091503268
Number of discordan

# Airport

In [76]:
dr_res_by_event_filepath = "/home/wmnlab/jackbedford/udp-reliability-research/handover_profiling/20240417_1333007d66_new_data_sync_v2/dr_20240417_1452001e84_anchor_by_event/dl_lost/results/A_20240418_1558000_2914d_iter2.pkl"
dr_res_by_packet_filepath = "/home/wmnlab/jackbedford/udp-reliability-research/handover_profiling/20240417_1333007d66_new_data_sync_v2/dr_20240417_1452001e84_anchor_by_packet/dl_lost/results/A_20240418_1600000_2f9af_iter2.pkl"
sr_res_filepath = "/home/wmnlab/jackbedford/udp-reliability-research/handover_profiling/20240417_1333007d66_new_data_sync_v2/sr/dl_lost/results/A_20240417_1444002_9abb6_iter5.pkl"
with open(dr_res_by_event_filepath, 'rb') as f:
    dr_res_by_event = pickle.load(f)
with open(dr_res_by_packet_filepath, 'rb') as f:
    dr_res_by_packet = pickle.load(f)
with open(sr_res_filepath, 'rb') as f:
    sr_res = pickle.load(f)

In [77]:
print('======= Single Radio =======')
devices = [item[5] for item in sr_res]
mean_values = [item[1] for item in sr_res]
ground_truths = [item[3] for item in sr_res]

print(devices)
print(ground_truths)
print(mean_values)

tau, p_value = kendalltau(ground_truths, mean_values)
print('tau:', tau)
print('p_value:', p_value)

discordant_pairs, concordant_pairs, equal_pairs, partial_eq_pairs, T_x, T_y, total_pairs = count_discordant_concordant_pairs(ground_truths, mean_values)
tau_ = calc_tau(discordant_pairs, concordant_pairs, equal_pairs, partial_eq_pairs, T_x, T_y, total_pairs)
print('tau self-calc:', tau_)

print("Number of discordant pairs:", discordant_pairs)
print("Number of concordant pairs:", concordant_pairs)
print("Number of equal pairs:", equal_pairs)
print("Number of partial equal pairs:", partial_eq_pairs)
print('Number of tied pairs in X:', T_x)
print('Number of tied pairs in Y:', T_y)
print('Number of total pairs:', total_pairs)
print()

print('======= Dual Radio By Event =======')
devices = [item[5] for item in dr_res_by_event]
mean_values = [item[1] for item in dr_res_by_event]
ground_truths = [item[3] for item in dr_res_by_event]

print(devices)
print(ground_truths)
print(mean_values)

tau, p_value = kendalltau(ground_truths, mean_values)
print('tau:', tau)
print('p_value:', p_value)

discordant_pairs, concordant_pairs, equal_pairs, partial_eq_pairs, T_x, T_y, total_pairs = count_discordant_concordant_pairs(ground_truths, mean_values)
tau_ = calc_tau(discordant_pairs, concordant_pairs, equal_pairs, partial_eq_pairs, T_x, T_y, total_pairs)
print('tau self-calc:', tau_)

print("Number of discordant pairs:", discordant_pairs)
print("Number of concordant pairs:", concordant_pairs)
print("Number of equal pairs:", equal_pairs)
print("Number of partial equal pairs:", partial_eq_pairs)
print('Number of tied pairs in X:', T_x)
print('Number of tied pairs in Y:', T_y)
print('Number of total pairs:', total_pairs)
print()

print('======= Dual Radio By Packet =======')
devices = [item[5] for item in dr_res_by_packet]
mean_values = [item[1] for item in dr_res_by_packet]
ground_truths = [item[3] for item in dr_res_by_packet]

print(devices)
print(ground_truths)
print(mean_values)

tau, p_value = kendalltau(ground_truths, mean_values)
print('tau:', tau)
print('p_value:', p_value)

discordant_pairs, concordant_pairs, equal_pairs, partial_eq_pairs, T_x, T_y, total_pairs = count_discordant_concordant_pairs(ground_truths, mean_values)
tau_ = calc_tau(discordant_pairs, concordant_pairs, equal_pairs, partial_eq_pairs, T_x, T_y, total_pairs)
print('tau self-calc:', tau_)

print("Number of discordant pairs:", discordant_pairs)
print("Number of concordant pairs:", concordant_pairs)
print("Number of equal pairs:", equal_pairs)
print("Number of partial equal pairs:", partial_eq_pairs)
print('Number of tied pairs in X:', T_x)
print('Number of tied pairs in Y:', T_y)
print('Number of total pairs:', total_pairs)
print()

======= Single Radio =======
['sm00', 'sm01', 'sm02', 'sm03', 'sm04', 'sm05', 'sm06', 'sm07', 'sm08', 'sm00', 'sm01', 'sm02', 'sm03', 'sm04', 'sm05', 'sm06', 'sm07', 'sm08']
[1.2739824667425692, 1.1163878571539843, 1.0289777263458912, 0.0, 0.8618540962410233, 1.1315151275750204, 0.8378746025520125, 0.6997329475909647, 0.19606206777605306, 1.2158015817159644, 0.7944186512296827, 0.6680148991746481, 0.7674919453681486, 0.6234509003207384, 0.9951524774773521, 0.9186526205689837, 0.9267759125054107, 0.164236135065305]
[1.0433070939169296, 0.9548956903029776, 0.9380137586478359, 0.1765571358509073, 0.8585715843516538, 1.0354140460020411, 0.7970564971908575, 0.78850932828901, 0.23131689097339292, 0.9499753813114946, 0.8251108768240316, 0.8154488529360221, 0.8182315361021647, 0.6537351325839238, 0.9623460681175297, 0.9404988089376504, 0.8858838308814775, 0.27189091872091653]
tau: 0.8431372549019609
p_value: 1.2272689725953054e-08
tau self-calc: 0.8431372549019608
Number of discordant pairs: 1